In [3]:
import warnings
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import(
    RandomForestClassifier,
    HistGradientBoostingClassifier
)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from scripts.wrapped import Wrapped
from scripts.train import(
    cross_validate_balancead,
    permutation,
    combination
)

In [4]:
warnings.filterwarnings("ignore")

wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

# Importando dados para treinamento

In [5]:
df = wp.load_data('df_instrumentos_features_selecionadas')
df

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,mfcc0,...,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,file_name
0,0.010380,0.038218,0.513887,0.581963,0.434641,0.293949,0.317826,0.271148,0.548258,-24.170067,...,3409.640722,2600.055158,1.098243,5260.821321,-44.640682,-30.202150,0.149672,33095,accordion,000139_119040.ogg
1,0.084643,-0.035337,0.175587,0.429345,0.840043,0.384524,0.282545,0.346341,0.482365,-90.556145,...,3516.595715,2385.271420,1.360742,5225.124166,-46.575737,-59.323132,0.112181,24785,accordion,000145_172800.ogg
2,-0.030127,0.082404,0.453814,0.219808,0.228484,0.446050,0.178916,0.250981,0.327460,-106.124260,...,3256.491175,2069.850731,1.343197,4287.155824,-56.276707,-71.512909,0.096007,21230,accordion,000201_168960.ogg
3,0.157274,-0.066099,0.209729,0.338755,0.660264,0.256625,0.275209,0.180417,0.399448,-66.319801,...,3855.791675,2903.979876,1.218963,6751.133809,-42.723160,-44.649315,0.144010,31836,accordion,000212_211200.ogg
4,-0.049096,-0.029067,0.484400,0.334572,0.441629,0.679485,0.382958,0.326033,0.205775,-226.263824,...,2883.225952,1251.374287,1.339527,2263.034730,-62.227947,-78.196365,0.059804,13214,accordion,000640_49920.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8471,0.070358,0.003048,0.332754,0.359639,0.473210,0.512169,0.607755,0.500391,0.378782,-87.742729,...,3664.812127,3194.922838,1.253166,6714.437434,-53.587788,-76.614616,0.134979,29855,voice,153341_184320.ogg
8472,0.006115,-0.030219,0.161793,0.190661,0.249882,0.378161,0.461520,0.332309,0.294171,-179.792145,...,2879.597305,1351.326334,1.450944,2412.643029,-52.071827,-76.029976,0.061211,13533,voice,153412_134400.ogg
8473,-0.008339,0.005653,0.589552,0.309548,0.187583,0.169132,0.240637,0.332375,0.333517,-153.614334,...,3274.875402,2685.831528,1.274003,4614.225695,-66.120743,-80.000000,0.157649,34869,voice,153478_30720.ogg
8474,0.108185,0.025366,0.615495,0.541314,0.467334,0.611199,0.791296,0.678383,0.508162,-80.250626,...,3376.494191,1428.726581,1.081204,3358.205447,-30.260586,-13.068514,0.032283,7136,voice,153986_203520.ogg


In [6]:
# instrumentos de corda:
inst_corda = ["cello", "guitar", "violin", "bass", "banjo", "mandolin","ukulele", "piano"]

# instrumentos de sopro:
inst_sopro = ["clarinet", "trombone", "flute", "trumpet", "saxophone"]

#instrumentos de percusão:
inst_percusao = ["mallet_percussion", "drums", "cymbals"]

# instrumentos aerofone
inst_aerofone = ["acordion", "organ"] 

# Digital - synthesizer e voice

In [7]:
le = LabelEncoder()
df['labels'] = le.fit_transform(df.instrumento)

# Teste 5: Permutação

- Models 

In [8]:
models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    MLPClassifier(),
    SVC(),
])

features = np.arange(0, 44)

### Instrumentos de corda

- filtro para dados de instruementos de corda

In [8]:
df_inst_corda = df[df["instrumento"].isin(inst_corda)]

- Split dados de treino e o target

In [9]:
X_corda = df.drop(columns=['labels', 'instrumento','file_name'])
y_corda = df['labels'].to_frame()

- Treinando modelos para instrumentos de corda

In [ ]:
acuracy_models = [cross_validate_balancead(k=5, model=model, X=X_corda, y=y_corda) for model in models]

dict_results = {
    'Naive Bayes': acuracy_models[0],
    'KNN': acuracy_models[1],
    'Arvore de Decisão': acuracy_models[2],
    'Floresta Aleatoria': acuracy_models[3],
    'HistGradientBoosting': acuracy_models[4],
    'LIGHTGBM': acuracy_models[5],
    'MLP': acuracy_models[6],
    'SVC': acuracy_models[7],
}

dict_results

In [2]:
comb_six = combination(features, 7)

NameError: name 'combination' is not defined

In [43]:
len(comb_six)

7059052

In [39]:
df_inst_corda.iloc[:, comb_six[0]]

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4
511,-0.126759,-0.059743,0.413241,0.216319,0.150052,0.123013
512,-0.031109,-0.064497,0.226724,0.379728,0.534193,0.384524
513,0.009446,0.006492,0.469337,0.108270,0.112574,0.109360
514,0.037982,-0.073602,0.340242,0.296036,0.302441,0.279111
515,0.062163,-0.025827,0.641626,0.313273,0.320937,0.305392
...,...,...,...,...,...,...
8019,0.051950,0.691044,0.773323,0.582712,0.532384,0.274606
8020,0.015595,-0.058762,0.120028,0.238263,0.234643,0.267743
8021,0.093624,-0.031124,0.253249,0.321181,0.309254,0.265595
8022,0.004588,-0.009204,0.124913,0.174113,0.185558,0.169614
